In [2]:
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews_all
import datetime as dt

In [3]:
%%time
#Fetch reviews for the US Market

us_reviews = reviews_all(
    'free.programming.programming',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'/For other markets us us lang_code like 'fr' for France
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

CPU times: total: 719 ms
Wall time: 4.96 s


In [4]:
#Convert JSON into Pandas DF

df_new = pd.json_normalize(us_reviews)

In [5]:
df_new.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOFBKIcD-NCuP0R7fS12raRTIYarfAV9kv_rYwq...,Ujef Malek,https://play-lh.googleusercontent.com/a-/AOh14...,Very good app for computer engineering student...,5,0,10.0.1.06,2022-06-12 12:38:38,None,NaT
1,gp:AOqpTOHgFacTuzYokVmU8xpSwdwjCRDGGe43ov0gNcq...,Utkarsh Sinha,https://play-lh.googleusercontent.com/a-/AOh14...,Nice,5,0,10.0.1.05,2022-06-12 11:20:14,None,NaT
2,gp:AOqpTOFs2WiCw_qfsji7XWFv4Tps_08p0HBVeKbbMFE...,Satyam Dahiwal,https://play-lh.googleusercontent.com/a-/AOh14...,"Since new update, cannot watch the course vide...",5,5,10.0.1.05,2022-06-10 14:39:08,"Hi, sorry for the inconvenience caused. This p...",2022-06-10 10:36:05
3,gp:AOqpTOFV1tyfXizdXh7BdruGQJg0IAdMSjI4bUIs-6m...,jatin singh,https://play-lh.googleusercontent.com/a-/AOh14...,App crashes as soon as I open the video lectur...,5,0,10.0.1.05,2022-06-10 11:08:18,"Hi, sorry for the inconvenience caused. This p...",2022-06-10 10:35:22
4,gp:AOqpTOHp2L1vwfaB_8VidcwSxcmajJvdDAB72xi3CEc...,James Zenos,https://play-lh.googleusercontent.com/a-/AOh14...,Excellent App,5,0,10.0.1.05,2022-06-10 10:57:53,"Hi, sorry for the inconvenience caused. This p...",2022-06-10 10:36:13


In [6]:
#Add two nee columns for Date and Time

df_new['date'] = df_new['at'].dt.strftime('%Y-%m-%d')
df_new['time'] = df_new['at'].dt.strftime('%H:%M:%S')

In [7]:
#Change content type to string before running 

df_new['content'] = df_new['content'].astype('str')

In [1]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

C:\Users\gagan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 1.32G/1.32G [00:59<00:00, 23.7MB/s]
Downloading: 100%|██████████| 256/256 [00:00<00:00, 64.1kB/s]
Downloading: 100%|██████████| 780k/780k [00:00<00:00, 1.27MB/s]
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 521kB/s]  
Downloading: 100%|██████████| 150/150 [00:00<00:00, 35.6kB/s]


In [8]:
#Sentiment function from transformer

df_new['result']=df_new['content'].apply(lambda x: sentiment_analysis(x))

In [9]:
df_new['sentiment']=df_new['result'].apply(lambda x: (x[0]['label']))

df_new['sentimentScore']=df_new['result'].apply(lambda x: x[0]['score'])

In [10]:
df_new.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,date,time,result,sentiment,sentimentScore
0,gp:AOqpTOFBKIcD-NCuP0R7fS12raRTIYarfAV9kv_rYwq...,Ujef Malek,https://play-lh.googleusercontent.com/a-/AOh14...,Very good app for computer engineering student...,5,0,10.0.1.06,2022-06-12 12:38:38,None,NaT,2022-06-12,12:38:38,"[{'label': 'POSITIVE', 'score': 0.998752593994...",POSITIVE,0.998753
1,gp:AOqpTOHgFacTuzYokVmU8xpSwdwjCRDGGe43ov0gNcq...,Utkarsh Sinha,https://play-lh.googleusercontent.com/a-/AOh14...,Nice,5,0,10.0.1.05,2022-06-12 11:20:14,None,NaT,2022-06-12,11:20:14,"[{'label': 'POSITIVE', 'score': 0.997769474983...",POSITIVE,0.997769
2,gp:AOqpTOFs2WiCw_qfsji7XWFv4Tps_08p0HBVeKbbMFE...,Satyam Dahiwal,https://play-lh.googleusercontent.com/a-/AOh14...,"Since new update, cannot watch the course vide...",5,5,10.0.1.05,2022-06-10 14:39:08,"Hi, sorry for the inconvenience caused. This p...",2022-06-10 10:36:05,2022-06-10,14:39:08,"[{'label': 'NEGATIVE', 'score': 0.991502404212...",NEGATIVE,0.991502
3,gp:AOqpTOFV1tyfXizdXh7BdruGQJg0IAdMSjI4bUIs-6m...,jatin singh,https://play-lh.googleusercontent.com/a-/AOh14...,App crashes as soon as I open the video lectur...,5,0,10.0.1.05,2022-06-10 11:08:18,"Hi, sorry for the inconvenience caused. This p...",2022-06-10 10:35:22,2022-06-10,11:08:18,"[{'label': 'NEGATIVE', 'score': 0.995246946811...",NEGATIVE,0.995247
4,gp:AOqpTOHp2L1vwfaB_8VidcwSxcmajJvdDAB72xi3CEc...,James Zenos,https://play-lh.googleusercontent.com/a-/AOh14...,Excellent App,5,0,10.0.1.05,2022-06-10 10:57:53,"Hi, sorry for the inconvenience caused. This p...",2022-06-10 10:36:13,2022-06-10,10:57:53,"[{'label': 'POSITIVE', 'score': 0.998693287372...",POSITIVE,0.998693


In [11]:
#Add also the length of the content
df_new['contentLength'] = df_new['content'].apply(lambda x: len(x.split(' ')))

In [12]:
#Add country of the reviews
df_new['country'] = 'US'

In [14]:
df_new.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'date', 'time', 'result', 'sentiment', 'sentimentScore',
       'contentLength', 'country'],
      dtype='object')

In [16]:
#Organize the rows

df_final = df_new[['reviewId',
              'date',
              'time', 
              'reviewCreatedVersion',
              'country',
              'score',
              'thumbsUpCount', 
              'content',
              'contentLength',
              'sentiment',
              'sentimentScore',
              'contentLength',
              'repliedAt',
              'replyContent']]

In [17]:
df_final.head()

,reviewId,date,time,reviewCreatedVersion,country,score,thumbsUpCount,content,contentLength,sentiment,sentimentScore,contentLength,repliedAt,replyContent
0,gp:AOqpTOFBKIcD-NCuP0R7fS12raRTIYarfAV9kv_rYwq...,2022-06-12,12:38:38,10.0.1.06,US,5,0,Very good app for computer engineering student...,11,POSITIVE,0.998753,11,NaT,None
1,gp:AOqpTOHgFacTuzYokVmU8xpSwdwjCRDGGe43ov0gNcq...,2022-06-12,11:20:14,10.0.1.05,US,5,0,Nice,1,POSITIVE,0.997769,1,NaT,None
2,gp:AOqpTOFs2WiCw_qfsji7XWFv4Tps_08p0HBVeKbbMFE...,2022-06-10,14:39:08,10.0.1.05,US,5,5,"Since new update, cannot watch the course vide...",26,NEGATIVE,0.991502,26,2022-06-10 10:36:05,"Hi, sorry for the inconvenience caused. This p..."
3,gp:AOqpTOFV1tyfXizdXh7BdruGQJg0IAdMSjI4bUIs-6m...,2022-06-10,11:08:18,10.0.1.05,US,5,0,App crashes as soon as I open the video lectur...,26,NEGATIVE,0.995247,26,2022-06-10 10:35:22,"Hi, sorry for the inconvenience caused. This p..."
4,gp:AOqpTOHp2L1vwfaB_8VidcwSxcmajJvdDAB72xi3CEc...,2022-06-10,10:57:53,10.0.1.05,US,5,0,Excellent App,2,POSITIVE,0.998693,2,2022-06-10 10:36:13,"Hi, sorry for the inconvenience caused. This p..."


In [19]:
df_final.to_csv(r'C:\Users\gagan\Python Projects\sentiment_analysis\sent.csv', index=False)